In [0]:
# Mount Google Driver
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
drive.mount(ROOT)           # we mount the google drive at /content/drive
# change to clrs directionary
%cd "/content/drive/My Drive/Colab Notebooks/fluent_python_notes"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/fluent_python_notes


In [0]:
%mkdir ch10
!touch ch10/__init__.py

mkdir: cannot create directory ‘ch10’: File exists


In [0]:
import imp

## 10.1 `Vector`类： 用户定义的序列类型

- 实现支持序列方法的多维向量类

## 10.2 `Vector` 类第 1 版： 与 `Vector2d` 类兼容

###### 示例 10-2　vector_v1.py

In [0]:
%%writefile ch10/vector_v1.py
from array import array
import reprlib
import math


class Vector:
  typecode = 'd'

  def __init__(self, components):
    self._components = array(self.typecode, components)  # 受保护的属性

  def __iter__(self):
    return iter(self._components)

  def __repr__(self):
    components = reprlib.repr(self._components)  # 获取 self._components 的有限长度表示
    components = components[components.find('['):-1]  # 去掉前面的 array( 'd 和后面的的 )
    return 'Vector({})'.format(components)
  
  def __str__(self):
    return str(tuple(self))  # 利用 Vector2d 是可迭代对象

  def __bytes__(self):
    return (bytes([ord(self.typecode)]) + 
        bytes(array(self.typecode, self)))

  def __eq__(self, other):
    return tuple(self) == tuple(other)

  def __abs__(self):
    return math.sqrt(sum(x * x for x in self))

  def __bool__(self):
    return bool(abs(self))

  def angle(self):
    return math.atan2(self.y, self.x)

  @classmethod
  def frombytes(cls, octets):
    typecode = chr(octets[0])
    memv = memoryview(octets[1:]).cast(typecode)
    return cls(memv)

Overwriting ch10/vector_v1.py


###### 示例 10-1 测试 `Vector.__init__` 和 `Vector.__repr__` 方法

In [0]:
import ch10.vector_v1
imp.reload(ch10.vector_v1)
from ch10.vector_v1 import Vector

In [0]:
Vector([3.1, 4.2])

Vector([3.1, 4.2])

In [0]:
Vector((3, 4, 5))

Vector([3.0, 4.0, 5.0])

In [0]:
Vector(range(10))

Vector([0.0, 1.0, 2.0, 3.0, 4.0, ...])

## 10.3 协议和鸭子类型

- 在 Python 中创建功能完善的序列类型无需使用继承，只需实现符合序列协议的方法
- 在面向对象编程中，协议是非正式的接口，只在文档中定义，在代码中不定义
- Python 的序列协议只需要 `__len__` 和 `__getitem__` 两个方法
  - 任何类，只要使用标准的签名和语义实现了这两个方法，就能用在任何期待序列的地方
  - 为了支持迭代，只需实现 `__getitem__` 方法，没必要提供 `__len__` 方法

## 10.4 `Vector` 类第 2 版： 可切片的序列

In [0]:
%%writefile -a ch10/vector_v1.py


  def __len__(self):
    return len(self._components)

  def __getitem__(self, index):
    return self._components[index]

Appending to ch10/vector_v1.py


In [0]:
import ch10.vector_v1
imp.reload(ch10.vector_v1)
from ch10.vector_v1 import Vector

In [0]:
v1 = Vector([3, 4, 5])
len(v1)

3

In [0]:
v1[0], v1[-1]

(3.0, 5.0)

In [0]:
v7 = Vector(range(7))
v7[1:4]

array('d', [1.0, 2.0, 3.0])

- 现在可以支持切片，不过尚不完美
- 如果 `Vector` 实例的切片也是 `Vector` 实例，而不是数组，那就更好了。对 `Vector` 来说，如果切片生成普通的数组，将会缺失大量功能

### 示例 10-4　了解 `__getitem__` 和切片的行为

In [0]:
class MySeq:
  def __getitem__(self, index):
    return index

In [0]:
s = MySeq()
s[1]

1

In [0]:
s[1:4]

slice(1, 4, None)

In [0]:
s[1:4:2]

slice(1, 4, 2)

- 如果 `[]` 中有括号，则 `__getitem__` 收到的是元组
- 元组中可以有多个切片对象

In [0]:
s[1:4:2, 9]

(slice(1, 4, 2), 9)

In [0]:
s[1:4:2, 7:9]

(slice(1, 4, 2), slice(7, 9, None))

###### 示例 10-5　查看 slice 类的属性

In [0]:
repr(slice)

"<class 'slice'>"

In [0]:
dir(slice)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'indices',
 'start',
 'step',
 'stop']

##### `indices` 方法

- `S.indices(len) -> (start, stop, stride)`
  > 给定长度为 `len` 的序列，计算 `S` 表示的扩展切片的起始（`start`）和结尾（`stop`）索引，以及步幅（`stride`）。超出边界的索引会被截掉，这与常规切片的处理方式一样  
- `indices` 方法开放了内置序列实现的棘手逻辑，用于优雅地处理缺失索引和负数索引，以及长度超过目标序列的切片
  - 这个方法会“整顿”元组，把 `start`、`stop` 和 `stride` 都变成非负数，而且都落在指定长度序列的边界内

In [0]:
slice(None, 10, 2).indices(5)

(0, 5, 2)

In [0]:
slice(-3, None, None).indices(5)

(2, 5, 1)

### 10.4.2 能处理切片的 `__getitem__` 方法

###### 示例 10-6　`vector_v2.py` 的部分代码

In [0]:
%%writefile ch10/vector_v2.py
from array import array
import reprlib
import math
import numbers


class Vector:
  typecode = 'd'

  def __init__(self, components):
    self._components = array(self.typecode, components)  # 受保护的属性

  def __iter__(self):
    return iter(self._components)

  def __repr__(self):
    components = reprlib.repr(self._components)  # 获取 self._components 的有限长度表示
    components = components[components.find('['):-1]  # 去掉前面的 array( 'd 和后面的的 )
    return 'Vector({})'.format(components)
  
  def __str__(self):
    return str(tuple(self))  # 利用 Vector2d 是可迭代对象

  def __bytes__(self):
    return (bytes([ord(self.typecode)]) + 
        bytes(array(self.typecode, self)))

  def __eq__(self, other):
    return tuple(self) == tuple(other)

  def __abs__(self):
    return math.sqrt(sum(x * x for x in self))

  def __bool__(self):
    return bool(abs(self))

  def angle(self):
    return math.atan2(self.y, self.x)

  @classmethod
  def frombytes(cls, octets):
    typecode = chr(octets[0])
    memv = memoryview(octets[1:]).cast(typecode)
    return cls(memv)

  def __len__(self):
    return len(self._components)

  def __getitem__(self, index):
    cls = type(self)  # 获取实例所属的类
    if isinstance(index, slice):
      return cls(self._components[index])
    elif isinstance(index, numbers.Integral):  # 如果 index 是 int 或其他整数类型
      return self._components[index]
    else:
      msg = '{cls.__name__} indices must be integers'
      raise TypeError(msg.format(cls=cls))


Writing ch10/vector_v2.py


- 大量使用 `isinstance` 可能表明面向对象设计得不好，不过在 `__getitem__` 方法中使用它处理切片是合理的

###### 示例 10-7　测试示例 10-6 中改进的 `Vector.__getitem__` 方法

In [0]:
import ch10.vector_v2
imp.reload(ch10.vector_v2)
from ch10.vector_v2 import Vector

In [0]:
v7 = Vector(range(7))  # 单个整数索引只获取一个分量
v7[-1]

6.0

In [0]:
v7[1:4]  # 切片索引创建一个新 Vecotr 实例

Vector([1.0, 2.0, 3.0])

In [0]:
v7[-1:]  # 长度为 1 的切片也创建一个 Vector 实例

Vector([6.0])

In [0]:
v7[1, 2]  # Vector 不支持多维索引，会抛出错误

TypeError: ignored

## 10.5 `Vector` 类第 3 版：动态存取属性 

- 属性查找失败后，解释器会调用 `__getattr__` 方法

###### 示例 10-8　vector_v3.py 的部分代码：在 vector_v2.py 中定义的 `Vector` 类里添加 `__getattr__` 方法

In [0]:
!cat ch10/vector_v2.py > ch10/vector_v3.py

In [0]:
%%writefile -a ch10/vector_v3.py


  shortcut_names = 'xyzt'
  
  def __getattr__(self, name):
    cls = type(self)
    if len(name) == 1:
      pos = cls.shortcut_names.find(name)  # find 也会查找 yz， 但是前一句已经限定 name 中只能有一个字符
      if 0 <= pos < len(self._components):
        return self._components[pos]
    msg = '{.__name__!r} object has no attribute {!r}'
    raise AttributeError(msg.format(cls, name))

Appending to ch10/vector_v3.py


###### 示例 10-9　不恰当的行为：为 v.x 赋值没有抛出错误，但是前后矛盾

In [0]:
import ch10.vector_v3
imp.reload(ch10.vector_v3)
from ch10.vector_v3 import Vector

In [0]:
v = Vector(range(5))
v

Vector([0.0, 1.0, 2.0, 3.0, 4.0])

In [0]:
v.x

0.0

In [0]:
v.x = 10
v.x

10

In [0]:
v

Vector([0.0, 1.0, 2.0, 3.0, 4.0])

###### 示例 10-10　vector_v3.py 的部分代码：在 `Vector` 类中实现 `__setattr__` 方法

In [0]:
%%writefile -a ch10/vector_v3.py


  def __setattr__(self, name, value):
    cls = type(self)
    if len(name) == 1:
      if name in cls.shortcut_names:  # 如果 name 是 xyzt 中的一个，设置特殊的错误消息
        error = 'readonly attribute {attr_name!r}'
      elif name.islower():  # 为所有的小写字母设置特殊的错误消息
        error = "can't set attributes 'a' to 'z' in {cls_name!r}"
      else:
        error = ''
  
      if error:
        msg = error.format(cls_name=cls.__name__, attr_name=name)
        raise AttributeError(msg)
    super().__setattr__(name, value)  # 在超类上调用 __setattr__ 方法，提供标准行为
  

Appending to ch10/vector_v3.py


In [0]:
import ch10.vector_v3
imp.reload(ch10.vector_v3)
from ch10.vector_v3 import Vector

In [0]:
v = Vector(range(5))
v.x

0.0

In [0]:
v.x = 10

AttributeError: ignored

In [0]:
v.p = 10

AttributeError: ignored

######  `__slots__` 属性

- `__slots__` 属性可以防止设置新实例属性
- 但是只为了避免创建实例属性没有必要使用 `__slots__`, `__slots__` 属性应该只用于节省内存，而且仅当内存严重不足时才应该这么做

###### 多数时候，如果实现了 `__getattr__` 方法，那么也需要定义相应的 `__setattr__` 方法，以防对象的行为不一致

## 10.6 `Vector` 类第 4 版： 散列和快速等值测试

###### `functools.reduce`

- 归约函数 `reduce` 的整体思路如下
  - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200427211242.png width=600>
- `reduce()` 函数的第一个参数是接收两个参数的函数（设为 `fn`）， 第二个参数是一个可迭代对象(设为一个列表`lst`)
  - 调用 `reduce(fn, lst)` 时， `fn` 会应用到第一对元素上，即 `fn(lst[0], lst[1])`， 生成第一个结果 `r1`
  - 然后，`fn` 会应用到 `r1` 和下一个元素上，即 `fn(r1, lst[2])`，生成第二个结果 `r2`
  - 接着，调用 `fn(r2,lst[3])`，生成 `r3`……直到最后一个元素，返回最后得到的结果 `rN`。 
- 使用 `reduce` 时最好提供第三个参数， `reduce(function, iterable, initializer)`
  - `initializer` 是序列为空时的返回结果。可以避免序列为空是，出现下述错误
    - `TypeError: reduce() of empyt sequence with no initial value`
  - 如果序列不为空，则在归约中使用 `initializer` 作为第一个参数，因此应该为恒等值
    - 对 `+`, `|` 和 `^` 来说， `initializer` 应该是 `0`， 而对 `*` 或 `&` 来说，应该是 `1`

- 使用 `reduce` 计算 $5!$

In [0]:
2 * 3 * 4 * 5

120

In [0]:
import functools
functools.reduce(lambda a, b: a*b, range(1, 6))

120

###### 示例 10-11 计算整数 0-5 的累计异或的 3 种方式

In [0]:
n = 0
for i in range(1, 6):
  n ^= i
n

1

In [0]:
import functools
functools.reduce(lambda a, b: a^b, range(6))

1

In [0]:
import operator
functools.reduce(operator.xor, range(6))

1

###### 示例 10-12 `vector_v4.py` ： 在 `vector_v3.py` 的基础上添加 `__hash__` 方法

In [0]:
%%writefile -a ch10/vector_v4.py
from array import array
import reprlib
import math
import numbers
import functools  # 为了使用 reduce 模块
import operator  # 为了使用 xor 函数


class Vector:
  typecode = 'd'

  def __init__(self, components):
    self._components = array(self.typecode, components)  # 受保护的属性

  def __iter__(self):
    return iter(self._components)

  def __repr__(self):
    components = reprlib.repr(self._components)  # 获取 self._components 的有限长度表示
    components = components[components.find('['):-1]  # 去掉前面的 array( 'd 和后面的的 )
    return 'Vector({})'.format(components)
  
  def __str__(self):
    return str(tuple(self))  # 利用 Vector2d 是可迭代对象

  def __bytes__(self):
    return (bytes([ord(self.typecode)]) + 
        bytes(array(self.typecode, self)))

  def __abs__(self):
    return math.sqrt(sum(x * x for x in self))

  def __bool__(self):
    return bool(abs(self))

  def angle(self):
    return math.atan2(self.y, self.x)

  @classmethod
  def frombytes(cls, octets):
    typecode = chr(octets[0])
    memv = memoryview(octets[1:]).cast(typecode)
    return cls(memv)

  def __len__(self):
    return len(self._components)

  def __getitem__(self, index):
    cls = type(self)  # 获取实例所属的类
    if isinstance(index, slice):
      return cls(self._components[index])
    elif isinstance(index, numbers.Integral):  # 如果 index 是 int 或其他整数类型
      return self._components[index]
    else:
      msg = '{cls.__name__} indices must be integers'
      raise TypeError(msg.format(cls=cls))

  shortcut_names = 'xyzt'
  
  def __getattr__(self, name):
    cls = type(self)
    if len(name) == 1:
      pos = cls.shortcut_names.find(name)  # find 也会查找 yz， 但是前一句已经限定 name 中只能有一个字符
      if 0 <= pos < len(self._components):
        return self._components[pos]
    msg = '{.__name__!r} object has no attribute {!r}'
    raise AttributeError(msg.format(cls, name))

  def __setattr__(self, name, value):
    cls = type(self)
    if len(name) == 1:
      if name in cls.shortcut_names:  # 如果 name 是 xyzt 中的一个，设置特殊的错误消息
        error = 'readonly attribute {attr_name!r}'
      elif name.islower():  # 为所有的小写字母设置特殊的错误消息
        error = "can't set attributes 'a' to 'z' in {cls_name!r}"
      else:
        error = ''
  
      if error:
        msg = error.format(cls_name=cls.__name__, attr_name=name)
        raise AttributeError(msg)
    super().__setattr__(name, value)  # 在超类上调用 __setattr__ 方法，提供标准行为

  def __eq__(self, other):
    return tuple(self) == tuple(other)

  def __hash__(self):
    hashes = (hash(x) for x in self._components)
    return functools.reduce(operator.xor, hashes, 0) # 0 是初始值，避免 hashes 为空

Writing ch10/vector_v4.py


- `__hash__` 方法的实现是一种映射归约
  - <img src=https://raw.githubusercontent.com/Lijunjie9502/PicBed/master/20200428071845.png width=600>
- 因此， `__hash__` 也可采用以下方式实现
  - ```python
    def __hash__(self):
      hashes = map(hash, self._components)
      return functools.reduce(operator.xor, hashes)
  ```

###### 示例 10-13 为了提高比较的效率，`Vector.__eq__` 方法在 `for` 循环中使用 `zip` 函数

- 之前的 `__eq__` 方法要完整的复制两个操作数，构造两个元组，目的是为了使用 `tuple` 类型的 `__eq__` 方法
- 如此当 `Vector` 实例中有几千个分量时，效率十分低下
```python
  def __eq__(self, other):
    if len(self) != len(other):  # 长度测试是有必要的，因为一旦 zip 中有一个输入耗尽， zip 函数会停止生成值，而且不发出警告
      return False
    for a, b in zip(self, other):
      if a != b:
        return False
    return True
```

###### 示例 10-14 使用 `zip` 和 `all` 函数实现 `Vector.__eq__` 方法，逻辑与示例 10-13 一样

```python
  def __eq__(self, other):
    return len(self) == len(other) and all(a == b for a, b in zip(self, other))
```

##### 出色的 `zip` 函数

- 使用 `zip` 函数能轻松地并行迭代两个或更多可迭代对象，返回的元组可以拆包为变量

###### 示例 10-15 `zip` 内置函数的使用示例

In [0]:
zip(range(3), 'ABC')  # 返回生成器，按需生成元组

In [0]:
list(zip(range(3), 'ABC'))

[(0, 'A'), (1, 'B'), (2, 'C')]

In [0]:
list(zip(range(3), 'ABC', [0.0, 1.1, 2.2, 3.3]))  # 当一个可迭代对象耗尽后，其不发出警告就停止

[(0, 'A', 0.0), (1, 'B', 1.1), (2, 'C', 2.2)]

- `itertools.zip_longest` 函数使用可选的 `fillvalue`（默认值为 `None`）填充缺失的值，因此可以持续产出，直到最长的可迭代对象耗尽

In [0]:
from itertools import zip_longest
list(zip_longest(range(3), 'ABC', [0.0, 1.1, 2.2, 3.3], fillvalue=-1))

[(0, 'A', 0.0), (1, 'B', 1.1), (2, 'C', 2.2), (-1, -1, 3.3)]

## 10.7 `Vector` 类第五版：格式化

- 使用 `h` 来表示球面坐标输出, [$n$ 维球体的球面坐标计算公式](https://en.wikipedia.org/wiki/N-sphere)

###### 示例 10-16 `vector_v5.py`

In [0]:
%%writefile ch10/vector_v5.py
from array import array
import reprlib
import math
import numbers
import functools  # 为了使用 reduce 模块
import operator  # 为了使用 xor 函数
import itertools  # 为了使用 chain 函数


class Vector:
  typecode = 'd'

  def __init__(self, components):
    self._components = array(self.typecode, components)  # 受保护的属性

  def __iter__(self):
    return iter(self._components)

  def __repr__(self):
    components = reprlib.repr(self._components)  # 获取 self._components 的有限长度表示
    components = components[components.find('['):-1]  # 去掉前面的 array( 'd 和后面的的 )
    return 'Vector({})'.format(components)
  
  def __str__(self):
    return str(tuple(self))  # 利用 Vector2d 是可迭代对象

  def __bytes__(self):
    return (bytes([ord(self.typecode)]) + 
        bytes(array(self.typecode, self)))

  def __eq__(self, other):
    return len(self) == len(other) and all(a == b for a, b in zip(self, other))

  def __hash__(self):
    hashes = (hash(x) for x in self._components)
    return functools.reduce(operator.xor, hashes, 0) # 0 是初始值，避免 hashes 为空

  def __abs__(self):
    return math.sqrt(sum(x * x for x in self))

  def __bool__(self):
    return bool(abs(self))

  def angle(self):
    return math.atan2(self.y, self.x)

  def __len__(self):
    return len(self._components)

  def __getitem__(self, index):
    cls = type(self)  # 获取实例所属的类
    if isinstance(index, slice):
      return cls(self._components[index])
    elif isinstance(index, numbers.Integral):  # 如果 index 是 int 或其他整数类型
      return self._components[index]
    else:
      msg = '{cls.__name__} indices must be integers'
      raise TypeError(msg.format(cls=cls))

  shortcut_names = 'xyzt'
  
  def __getattr__(self, name):
    cls = type(self)
    if len(name) == 1:
      pos = cls.shortcut_names.find(name)  # find 也会查找 yz， 但是前一句已经限定 name 中只能有一个字符
      if 0 <= pos < len(self._components):
        return self._components[pos]
    msg = '{.__name__!r} object has no attribute {!r}'
    raise AttributeError(msg.format(cls, name))

  def __setattr__(self, name, value):
    cls = type(self)
    if len(name) == 1:
      if name in cls.shortcut_names:  # 如果 name 是 xyzt 中的一个，设置特殊的错误消息
        error = 'readonly attribute {attr_name!r}'
      elif name.islower():  # 为所有的小写字母设置特殊的错误消息
        error = "can't set attributes 'a' to 'z' in {cls_name!r}"
      else:
        error = ''
  
      if error:
        msg = error.format(cls_name=cls.__name__, attr_name=name)
        raise AttributeError(msg)
    super().__setattr__(name, value)  # 在超类上调用 __setattr__ 方法，提供标准行为

  def angle(self, n):
    r = math.sqrt(sum(x*x for x in self[n:]))
    a = math.atan2(r, self[n-1])
    if (n == len(self) - 1) and (self[-1] < 0):
      return math.pi * 2 - a
    else:
      return a

  def angles(self):
    return (self.angle(n) for n in range(1, len(self)))

  def __format__(self, fmt_spec=''):
    if fmt_spec.endswith('h'):  # 球面坐标
      fmt_spec = fmt_spec[:-1]
      coords = itertools.chain([abs(self)], self.angles())  # 生成生成器表达式
      outer_fmt = '<{}>'
    else:
      coords = self
      outer_fmt = '({})'
    components = (format(c, fmt_spec) for c in coords)
    return outer_fmt.format(', '.join(components))

  @classmethod
  def frombytes(cls, octets):
    typecode = chr(octets[0])
    memv = memoryview(octets[1:]).cast(typecode)
    return cls(memv) 

Overwriting ch10/vector_v5.py


###### 测试球面坐标系

In [0]:
import ch10.vector_v5
imp.reload(ch10.vector_v5)
from ch10.vector_v5 import Vector

In [0]:
format(Vector([-1, -1, -1, -1]), 'h')

'<2.0, 2.0943951023931957, 2.186276035465284, 3.9269908169872414>'

In [0]:
format(Vector([2]*4), '3eh')

'<4.000000e+00, 1.047198e+00, 9.553166e-01, 7.853982e-01>'

In [0]:
format(Vector([0, 1, 0, 0]), '0.5fh')

'<1.00000, 1.57080, 0.00000, 0.00000>'